In [1]:
import torch
from torchvision import models

def load_trained_mobilenetv2(weights_path, num_classes):
    """
    加载已经训练好的 MobileNetV2 模型。

    参数：
    weights_path (str): 模型权重文件的路径（如 'mobilenetv2_epoch10.pth'）
    num_classes (int): 分类任务中的类别数量，应与训练时保持一致

    返回：
    model (torch.nn.Module): 加载了权重的模型，准备好用于推理或继续训练
    """
    # 初始化 MobileNetV2 结构，取消预训练，便于加载自己的权重
    model = models.mobilenet_v2(pretrained=False)

    # 替换分类器，确保与训练时一致（通常是根据数据集重新定义）
    model.classifier[1] = torch.nn.Linear(model.last_channel, num_classes)

    # 加载训练好的参数（state_dict）
    model.load_state_dict(torch.load(weights_path, map_location=torch.device("cpu")))

    # 将模型设置为评估模式，适用于推理阶段
    model.eval()

    return model

# model = load_trained_mobilenetv2("mobilenetv2_epoch10.pth", num_classes=38)


In [5]:
model = load_trained_mobilenetv2("mobilenetv2_epoch10.pth", num_classes=39)

In [7]:
from PIL import Image
from torchvision import transforms
import torch

def predict_image(model, image_path, class_names):
    """
    对单张图像进行分类预测。

    参数：
    model (nn.Module): 加载好的模型
    image_path (str): 图像文件路径
    class_names (list): 类别名称列表，索引对应模型输出的类别

    返回：
    predicted_class (str): 预测的类别名称
    """
    # 定义与训练一致的图像预处理操作
    preprocess = transforms.Compose([
        transforms.Resize((224, 224)),        # 调整图像大小
        transforms.ToTensor(),                # 转为张量
    ])

    image = Image.open(image_path).convert('RGB')
    image_tensor = preprocess(image)                 # 预处理图像
    image_tensor = image_tensor.unsqueeze(0)         # 增加 batch 维度 -> [1, C, H, W]

    model.eval()
    with torch.no_grad():
        outputs = model(image_tensor)
        _, predicted = torch.max(outputs, 1)         # 获取预测类别索引

    predicted_index = predicted.item()
    predicted_class = class_names[predicted_index]   # 索引对应类别名称
    return predicted_class


In [9]:
class_names = [
    'Apple___Apple_scab',  # 苹果黑星病 Apple scab
    'Apple___Black_rot',  # 苹果黑腐病 Apple Black rot
    'Apple___Cedar_apple_rust',  # 苹果雪松锈病 Apple Cedar apple rust
    'Apple___healthy',  # 苹果健康 Apple healthy
    'Blueberry___healthy',  # 蓝莓健康 Blueberry healthy
    'Cherry_(including_sour)___Powdery_mildew',  # 樱桃白粉病 Cherry Powdery mildew
    'Cherry_(including_sour)___healthy',  # 樱桃健康 Cherry healthy
    'Corn_(maize)___Cercospora_leaf_spot Gray_leaf_spot',  # 玉米灰斑病 Corn Gray leaf spot
    'Corn_(maize)___Common_rust_',  # 玉米普通锈病 Corn Common rust
    'Corn_(maize)___Northern_Leaf_Blight',  # 玉米北方叶斑病 Corn Northern Leaf Blight
    'Corn_(maize)___healthy',  # 玉米健康 Corn healthy
    'Grape___Black_rot',  # 葡萄黑腐病 Grape Black rot
    'Grape___Esca_(Black_Measles)',  # 葡萄腐烂病 Grape Esca (Black Measles)
    'Grape___Leaf_blight_(Isariopsis_Leaf_Spot)',  # 葡萄叶枯病 Grape Leaf blight (Isariopsis Leaf Spot)
    'Grape___healthy',  # 葡萄健康 Grape healthy
    'Orange___Haunglongbing_(Citrus_greening)',  # 橙黄龙病 Orange Huanglongbing (Citrus greening)
    'Peach___Bacterial_spot',  # 桃细菌性斑点 Peach Bacterial spot
    'Peach___healthy',  # 桃健康 Peach healthy
    'Pepper,_bell___Bacterial_spot',  # 灯笼椒细菌性斑点 Pepper, bell Bacterial spot
    'Pepper,_bell___healthy',  # 灯笼椒健康 Pepper, bell healthy
    'Potato___Early_blight',  # 马铃薯早疫病 Potato Early blight
    'Potato___Late_blight',  # 马铃薯晚疫病 Potato Late blight
    'Potato___healthy',  # 马铃薯健康 Potato healthy
    'Raspberry___healthy',  # 树莓健康 Raspberry healthy
    'Soybean___healthy',  # 大豆健康 Soybean healthy
    'Squash___Powdery_mildew',  # 南瓜白粉病 Squash Powdery mildew
    'Strawberry___Leaf_scorch',  # 草莓叶灼病 Strawberry Leaf scorch
    'Strawberry___healthy',  # 草莓健康 Strawberry healthy
    'Tomato___Bacterial_spot',  # 番茄细菌性斑点 Tomato Bacterial spot
    'Tomato___Early_blight',  # 番茄早疫病 Tomato Early blight
    'Tomato___Late_blight',  # 番茄晚疫病 Tomato Late blight
    'Tomato___Leaf_Mold',  # 番茄叶霉病 Tomato Leaf Mold
    'Tomato___Septoria_leaf_spot',  # 番茄叶斑病 Tomato Septoria leaf spot
    'Tomato___Spider_mites Two-spotted_spider_mite',  # 番茄二斑叶螨 Tomato Spider mites (Two-spotted spider mite)
    'Tomato___Target_Spot',  # 番茄靶斑病 Tomato Target Spot
    'Tomato___Tomato_Yellow_Leaf_Curl_Virus',  # 番茄黄化卷叶病毒病 Tomato Yellow Leaf Curl Virus
    'Tomato___Tomato_mosaic_virus',  # 番茄花叶病毒病 Tomato mosaic virus
    'Tomato___healthy',  # 番茄健康 Tomato healthy
    'background'  # 背景背景（非植物）Background (non-plant)
]


In [15]:

pred = predict_image(model, "apple_black_rot.png", class_names)
print("result：", pred)

result： Apple___Black_rot


In [19]:
pred = predict_image(model, "Apple_scab.png", class_names)
print("result：", pred)

result： Apple___Apple_scab


In [23]:
pred = predict_image(model, "Corn_Northern.png", class_names)
print("result：", pred)

result： Corn_(maize)___Northern_Leaf_Blight


In [25]:
pred = predict_image(model, "peach_bacterial.png", class_names)
print("result：", pred)

result： Peach___Bacterial_spot
